In [83]:
import datetime
import json
import requests

import pandas as pd

from bs4 import BeautifulSoup

### Making your first request

In [56]:
url = 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/agenda-do-presidente-da-republica/2021-07-13'
response = requests.get(url)

print("Response Object: ", response)
print("Response Status Code: ", response.status_code)
print("Response URL: ", response.url, "\n\n")
print("Request Headers :", response.request.headers, "\n\n")
print("Response Headers: ", response.headers)

Response Object:  <Response [200]>
Response Status Code:  200
Response URL:  https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/agenda-do-presidente-da-republica/2021-07-13 


Request Headers : {'User-Agent': 'python-requests/2.22.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'} 


Response Headers:  {'Date': 'Tue, 13 Jul 2021 21:14:49 GMT', 'Server': 'Zope/(2.13.28, python 2.7.16, linux2) ZServer/1.1', 'X-Cache-Operation': 'plone.app.caching.moderateCaching', 'Content-Language': 'pt-br', 'Expires': 'Sat, 16 Jul 2011 21:14:49 GMT', 'Vary': 'X-Anonymous,Accept-Encoding', 'X-Ua-Compatible': 'IE=edge,chrome=1', 'X-Cache-Rule': 'brasil.gov.agenda.AgendaDiaria', 'X-Frame-Options': 'DENY', 'Content-Type': 'text/html;charset=utf-8', 'cache-control': 'max-age=10, s-maxage=0, must-revalidate', 'X-Varnish': '9837696 5245768', 'Via': '1.1 varnish (Varnish/6.0), 1.1 www.gov.br', 'X-Cache': 'HIT', 'X-Varnish-Age': '860', 'Age': '0', 'Accept-Ranges': 'bytes', 

### Retrieving the page's content

In [96]:
#response.content

### Using BeautifulSoup

In [79]:
start_date = '2021-07-13'
url = 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/agenda-do-presidente-da-republica/{sd}'.format(sd=start_date)
response = requests.get(url)

In [80]:
soup = BeautifulSoup(response.content)
compromissos_tags = soup.findAll('li', class_='item-compromisso-wrapper')

In [81]:
compromisso_lista = []
for comp in compromissos_tags:
    compromisso = {}
    compromisso['titulo'] = comp.find('h4', class_='compromisso-titulo').text
    compromisso['start_hour'] = comp.find('time', class_='compromisso-inicio').text
    compromisso['end_hour'] = comp.find('time', class_='compromisso-fim').text
    compromisso['location'] = comp.find('div', class_='compromisso-local').text
    compromisso['reference_date'] = start_date
    compromisso_lista.append(compromisso)

    
compromisso_lista

[{'titulo': 'Luiz Eduardo Ramos, Ministro de Estado Chefe da Casa Civil da Presidência da República',
  'start_hour': '10h00',
  'end_hour': '10h30',
  'location': 'Palácio do Planalto',
  'reference_date': '2021-07-13'},
 {'titulo': 'Senador Eduardo Gomes (MDB/TO), Vice-Líder do Governo no Senado Federal; e Senador Jayme Campos (DEM/MT)',
  'start_hour': '14h30',
  'end_hour': '15h00',
  'location': 'Palácio do Planalto',
  'reference_date': '2021-07-13'},
 {'titulo': 'Paulo Guedes, Ministro de Estado da Economia',
  'start_hour': '15h30',
  'end_hour': '16h00',
  'location': 'Palácio do Planalto',
  'reference_date': '2021-07-13'},
 {'titulo': 'Solenidade alusiva à Sanção da Lei de Capitalização da Eletrobras',
  'start_hour': '16h30',
  'end_hour': '17h30',
  'location': 'Palácio do Planalto',
  'reference_date': '2021-07-13'}]

In [82]:
df = pd.DataFrame(compromisso_lista)
df

,titulo,start_hour,end_hour,location,reference_date
0,"Luiz Eduardo Ramos, Ministro de Estado Chefe d...",10h00,10h30,Palácio do Planalto,2021-07-13
1,"Senador Eduardo Gomes (MDB/TO), Vice-Líder do ...",14h30,15h00,Palácio do Planalto,2021-07-13
2,"Paulo Guedes, Ministro de Estado da Economia",15h30,16h00,Palácio do Planalto,2021-07-13
3,Solenidade alusiva à Sanção da Lei de Capitali...,16h30,17h30,Palácio do Planalto,2021-07-13


In [93]:
start = datetime.datetime.strptime("2021-07-05", "%Y-%m-%d")
end = datetime.datetime.strptime("2021-07-13", "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days + 1)]
date_range = [datetime.datetime.strftime(i, "%Y-%m-%d") for i in date_generated]

In [94]:
compromissos_lifetime = []
for d in date_range:
    try:
        url = 'https://www.gov.br/planalto/pt-br/acompanhe-o-planalto/agenda-do-presidente-da-republica/{sd}'.format(sd=d)
        response = requests.get(url)
        soup = BeautifulSoup(response.content)
        compromissos_tags = soup.findAll('li', class_='item-compromisso-wrapper')
        compromisso_lista = []
        for comp in compromissos_tags:
            try:
                compromisso = {}
                compromisso['titulo'] = comp.find('h4', class_='compromisso-titulo').text
                compromisso['start_hour'] = comp.find('time', class_='compromisso-inicio').text
                compromisso['end_hour'] = comp.find('time', class_='compromisso-fim').text
                compromisso['location'] = comp.find('div', class_='compromisso-local').text
                compromisso['reference_date'] = d
                compromisso_lista.append(compromisso)
            except:
                pass
        compromissos_lifetime.extend(compromisso_lista)
    except:
        pass


In [95]:
df = pd.DataFrame(compromissos_lifetime)
df

,titulo,start_hour,end_hour,location,reference_date
0,"André Mendonça, Advogado-Geral da União",10h00,10h30,Palácio do Planalto,2021-07-13
1,"Gustavo Montezano, Presidente do Banco Naciona...",11h30,12h00,Palácio do Planalto,2021-07-13
2,"Marcos Pontes, Ministro de Estado da Ciência, ...",14h00,14h30,Palácio do Planalto,2021-07-13
3,"Joaquim Alvaro Pereira Leite, Ministro de Esta...",14h30,15h00,Palácio do Planalto,2021-07-13
4,"Humberto Fernandes de Moura, Subchefe para Ass...",15h00,15h30,Palácio do Planalto,2021-07-13
5,"João Roma, Ministro de Estado da Cidadania",17h00,18h00,Palácio do Planalto,2021-07-13
6,Hasteamento da Bandeira Nacional,08h00,08h15,Palácio da Alvorada,2021-07-13
7,Reunião com Ministros de Estado,08h15,11h30,Palácio da Alvorada,2021-07-13
8,"Milton Ribeiro, Ministro de Estado da Educação",11h50,12h50,Palácio do Planalto,2021-07-13
9,"Deputado Cezinha de Madureira (PSD/SP), Vice-L...",15h55,16h10,Palácio do Planalto,2021-07-13
